In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

zcta = pd.read_csv('Maryland_Census_Data_-_ZIP_Code_Tabulation_Areas__ZCTAs_.csv')
zipcodes = gpd.read_file('zipcode.geojson')
neighs = gpd.read_file('Maryland_Baltimore_City_Neighborhoods.geojson')
mhinc = pd.read_csv('MASTER_MERGED.csv')
covid = pd.read_csv('COVID_Cleaned_Transposed.csv')
pops = pd.read_csv('PopsByZip.csv')

sjoined = gpd.sjoin(neighs, zipcodes, how='inner',op='intersects')

sjoined.to_csv('spatialJoin.csv')
#zipCleaned = pd.read_csv('ZipData_CLEANED.csv')
zipsList = [21227,21207,21230,21251,21229,21237,21287,21231,21226,21206,21222,21225,21211,21208,\
            21205,21218,21234,21202,21201,21213,21210,21209,21216,21236,21217,21224,21215,21212,\
                21223,21214,21239,21228]

print(zcta.columns)

Index(['OBJECTID_1', 'ZCTA5CE10', 'FIRST_STAT', 'FIRST_GEOI', 'FIRST_CLAS',
       'FIRST_MTFC', 'FIRST_FUNC', 'ZCTA5N', 'STATE', 'AREALAND', 'AREAWATR',
       'POP100', 'HU100', 'NHW', 'NHB', 'NHAI', 'NHA', 'NHNH', 'NHO', 'NHT',
       'HISP', 'PNHW', 'PNHB', 'PNHAI', 'PNHA', 'PNHNH', 'PNHO', 'PNHT',
       'PHISP', 'POP65_', 'PCTPOP65_', 'MEDAGE', 'VACNS', 'PVACNS', 'PHOWN',
       'PWOMORT', 'PRENT', 'PLT18SP', 'REPORT_2_P', 'REPORT_9_P'],
      dtype='object')


In [2]:
sjoined = sjoined[['LABEL','zipcode1']]
sjoined = sjoined.rename(columns={'LABEL':'Neigh','zipcode1':'Zip Code'})
mhinc = mhinc[['Neigh','CSA','MHINC','Pop2010']]

merged = pd.merge_ordered(mhinc, sjoined, fill_method='none')
#print(merged[['CSA','Pop2010']])
merged['Zip Code'] = merged['Zip Code'].astype(int)
merged = merged.sort_values(by=['Zip Code'])
print(merged.head())

            Neigh                                    CSA     MHINC  Pop2010  \
852     Poppleton  Poppleton/The Terraces/Hollins Market   20365.8     5086   
668  Mount Vernon                                Midtown   52477.2    15020   
31   Barre Circle             Washington Village/Pigtown   48273.2     5503   
33   Barre Circle             Washington Village/Pigtown   48273.2     5503   
762     Otterbein              Inner Harbor/Federal Hill  100932.2    12855   

     Zip Code  
852     21201  
668     21201  
31      21201  
33      21201  
762     21201  


In [3]:
incomeMeans = []
zipsList.sort()

for zipcode in zipsList: 
    df1 = merged[merged['Zip Code'] == zipcode]
    incMean = df1['MHINC'].unique().mean()
    incomeMeans.append(incMean)

In [4]:
latestCases = pd.DataFrame()
covid = covid.sort_values(by=['Zip Code'])
latestCases['Zip Code'] = covid['Zip Code'].astype(int)
latestCases['MHINC'] = incomeMeans
#latestCases['Pop2010'] = popSums
latestCases['TotalCases'] = covid.iloc[:,-1].astype(int)
latestCases = pd.merge_ordered(latestCases,pops,fill_method='none')
latestCases = latestCases.drop('Unnamed: 0',axis=1)
print(latestCases)

    Zip Code         MHINC  TotalCases  Pop2010
0      21201  48044.111111         468    16972
1      21202  53117.988889         851    22832
2      21205  38678.157143         686    16146
3      21206  53439.028571        2029    50846
4      21207  47495.616667        1804    48133
5      21208  49056.350000        1443    33917
6      21209  64002.133333         768    26465
7      21210  82454.580000         120    14292
8      21211  56940.450000         290    17351
9      21212  73300.960000         839    32322
10     21213  38321.400000        1140    32733
11     21214  59342.820000         621    20564
12     21215  47794.718182        2287    60161
13     21216  39209.887500         933    32071
14     21217  37689.862500         921    37111
15     21218  52487.736364        1302    49796
16     21222  36666.300000        2084    55786
17     21223  38385.090000         790    26366
18     21224  71251.371429        2630    49134
19     21225  32534.400000        1070  

In [5]:
zcta = zcta.rename(columns={'ZCTA5N':'Zip Code','POP100':'Pop2010','PCTPOP65_':'PCTPOP65'})
zcta = zcta.iloc[:,7:38]
zcta = zcta.drop(columns={'STATE'})
print(zcta)

     Zip Code   AREALAND  AREAWATR  Pop2010  HU100    NHW    NHB  NHAI   NHA  \
0       20601  115635266    387684    24156   8722   9785  11146   155   880   
1       20602   35830723    352762    24955   9736   8466  13054   116   731   
2       20603   44239637    219356    28967  10317   9625  15025    98  1446   
3       20606    7501011   1248760      431    230    377     45     1     4   
4       20607   54357590    448221     9802   3504   2165   6321    33   560   
..        ...        ...       ...      ...    ...    ...    ...   ...   ...   
463     21918   45741330   6628177     4284   1589   4096     55     6    30   
464     21919  112736603  22553121     3467   2179   3321     43    14     8   
465     21920    1290392         0      257    100    228     17     0     0   
466     21921  244939298  14098236    44471  17232  36707   3953    98   673   
467     21930    1907039    186920       94     58     67     21     0     1   

     NHNH  ...  PHISP  POP65_  PCTPOP65

In [6]:
zcta = zcta[zcta['Zip Code'].isin(zipsList)]
zcta.reset_index(drop=True,inplace=True)
zcta = zcta.append({'Zip Code':21287,'PCTPOP65':0,'Pop2010':0},ignore_index=True)
zcta = zcta.fillna(0)
print(zcta)

    Zip Code    AREALAND    AREAWATR  Pop2010    HU100      NHW      NHB  \
0    21201.0   3340677.0         0.0  16972.0  11255.0   5482.0   8896.0   
1    21202.0   4111039.0    331316.0  22832.0  10372.0   6049.0  14845.0   
2    21205.0   5304530.0         0.0  16146.0   7059.0   2933.0  11259.0   
3    21206.0  18667230.0      3869.0  50846.0  21762.0  13133.0  34557.0   
4    21207.0  26710727.0     81229.0  48133.0  20272.0   4683.0  40270.0   
5    21208.0  31925959.0     65741.0  33917.0  15621.0  18227.0  13209.0   
6    21209.0  18354471.0    101926.0  26465.0  12054.0  19694.0   3547.0   
7    21210.0   8579050.0      5635.0  14292.0   5949.0  11158.0    909.0   
8    21211.0   7219042.0     25184.0  17351.0   9287.0  13144.0   2299.0   
9    21212.0  12042941.0     97316.0  32322.0  14380.0  16979.0  12620.0   
10   21213.0   8833257.0      1899.0  32733.0  14968.0   1954.0  29824.0   
11   21214.0   7394837.0         0.0  20564.0   8981.0   8276.0  10950.0   
12   21215.0

In [7]:

zcta['Zip Code'] = zcta['Zip Code'].astype(int)

print(zcta)

    Zip Code    AREALAND    AREAWATR  Pop2010    HU100      NHW      NHB  \
0      21201   3340677.0         0.0  16972.0  11255.0   5482.0   8896.0   
1      21202   4111039.0    331316.0  22832.0  10372.0   6049.0  14845.0   
2      21205   5304530.0         0.0  16146.0   7059.0   2933.0  11259.0   
3      21206  18667230.0      3869.0  50846.0  21762.0  13133.0  34557.0   
4      21207  26710727.0     81229.0  48133.0  20272.0   4683.0  40270.0   
5      21208  31925959.0     65741.0  33917.0  15621.0  18227.0  13209.0   
6      21209  18354471.0    101926.0  26465.0  12054.0  19694.0   3547.0   
7      21210   8579050.0      5635.0  14292.0   5949.0  11158.0    909.0   
8      21211   7219042.0     25184.0  17351.0   9287.0  13144.0   2299.0   
9      21212  12042941.0     97316.0  32322.0  14380.0  16979.0  12620.0   
10     21213   8833257.0      1899.0  32733.0  14968.0   1954.0  29824.0   
11     21214   7394837.0         0.0  20564.0   8981.0   8276.0  10950.0   
12     21215

In [8]:
print(latestCases)

    Zip Code         MHINC  TotalCases  Pop2010
0      21201  48044.111111         468    16972
1      21202  53117.988889         851    22832
2      21205  38678.157143         686    16146
3      21206  53439.028571        2029    50846
4      21207  47495.616667        1804    48133
5      21208  49056.350000        1443    33917
6      21209  64002.133333         768    26465
7      21210  82454.580000         120    14292
8      21211  56940.450000         290    17351
9      21212  73300.960000         839    32322
10     21213  38321.400000        1140    32733
11     21214  59342.820000         621    20564
12     21215  47794.718182        2287    60161
13     21216  39209.887500         933    32071
14     21217  37689.862500         921    37111
15     21218  52487.736364        1302    49796
16     21222  36666.300000        2084    55786
17     21223  38385.090000         790    26366
18     21224  71251.371429        2630    49134
19     21225  32534.400000        1070  

In [9]:
zipData = pd.merge_ordered(zcta,latestCases,fill_method='none')
print(zipData)

    Zip Code    AREALAND    AREAWATR  Pop2010    HU100      NHW      NHB  \
0      21201   3340677.0         0.0  16972.0  11255.0   5482.0   8896.0   
1      21202   4111039.0    331316.0  22832.0  10372.0   6049.0  14845.0   
2      21205   5304530.0         0.0  16146.0   7059.0   2933.0  11259.0   
3      21206  18667230.0      3869.0  50846.0  21762.0  13133.0  34557.0   
4      21207  26710727.0     81229.0  48133.0  20272.0   4683.0  40270.0   
5      21208  31925959.0     65741.0  33917.0  15621.0  18227.0  13209.0   
6      21209  18354471.0    101926.0  26465.0  12054.0  19694.0   3547.0   
7      21210   8579050.0      5635.0  14292.0   5949.0  11158.0    909.0   
8      21211   7219042.0     25184.0  17351.0   9287.0  13144.0   2299.0   
9      21212  12042941.0     97316.0  32322.0  14380.0  16979.0  12620.0   
10     21213   8833257.0      1899.0  32733.0  14968.0   1954.0  29824.0   
11     21214   7394837.0         0.0  20564.0   8981.0   8276.0  10950.0   
12     21215

In [10]:
zipData.to_csv('ZipData_CLEANED.csv')